In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score
import scipy.stats as stats
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from datetime import datetime
from category_encoders import OrdinalEncoder, TargetEncoder
from catboost import CatBoostClassifier, CatBoostRegressor
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Functions

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    roc = roc_auc_score(y_test, predictions)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%'.format(accuracy))
    print(f'AUC = {roc}')
    return accuracy

In [3]:
train = pd.read_csv('../Data/training_set_features.csv', index_col='respondent_id')
test = pd.read_csv('../Data/test_set_features.csv', index_col ='respondent_id')
labels = pd.read_csv('../Data/training_set_labels.csv', index_col='respondent_id')

In [4]:
train.loc[(train['age_group'] == '65+ Years') & (train['employment_status'].isnull()), 'employment_status'] = 'Not in Labor Force'

In [5]:
num_cols = list(train.select_dtypes('number').columns)

cat_cols = [
    'race',
    'sex',
    'marital_status',
    'rent_or_own',
    'hhs_geo_region',
    'census_msa',
    'employment_industry',
    'employment_occupation'
]

ord_cols = [
    'age_group',
    'education',
    'income_poverty',
    'employment_status'
]




#Impute Train
for col in num_cols:
    train[col] = train[col].fillna(value=-1)
    test[col] = test[col].fillna(value=-1)

for col in (cat_cols + ord_cols):
    train[col] = train[col].fillna(value='None')
    test[col] = test[col].fillna(value='None')
test_labels = labels.copy()    

In [6]:
train['age_group'] = train['age_group'].map({
    '18 - 34 Years': 1,
    '35 - 44 Years': 2,
    '45 - 54 Years': 3,
    '55 - 64 Years': 4,
    '65+ Years': 5
})

train['education'] = train['education'].map({
    '< 12 Years': 1,
    '12 Years': 2,
    'Some College': 3,
    'College Graduate': 4,
    'None': -1
})

train['income_poverty'] = train['income_poverty'].map({
    'None': -1,
    'Below Poverty': 1,
    '<= $75,000, Above Poverty': 2,
    '> $75,000': 3
})

train['employment_status'] = train['employment_status'].map({
    'None': -1,
    'Unemployed': 1,
    'Employed': 2,
    'Not in Labor Force': 3
})




test['education'] = test['education'].map({
    '< 12 Years': 1,
    '12 Years': 2,
    'Some College': 3,
    'College Graduate': 4,
    'None': -1
})

test['income_poverty'] = test['income_poverty'].map({
    'None': -1,
    'Below Poverty': 1,
    '<= $75,000, Above Poverty': 2,
    '> $75,000': 3
})

test['employment_status'] = test['employment_status'].map({
    'None': -1,
    'Unemployed': 1,
    'Employed': 2,
    'Not in Labor Force': 3
})

In [7]:
for x in train[ord_cols].columns:
    print(x, train[x].unique())

age_group [4 2 1 5 3]
education [ 1  2  4  3 -1]
income_poverty [ 1  2  3 -1]
employment_status [ 3  2  1 -1]


In [8]:
all_cols = train.columns

train_test = train.copy()

In [9]:
h1n1_labels = labels[['h1n1_vaccine']]
seas_labels = labels[['seasonal_vaccine']]

# Transformation

In [10]:
cat_cols = train.select_dtypes('object').columns

In [11]:
h1n1_train = train.copy()
seas_train = train.copy()

h1n1_scaler = StandardScaler()
h1n1_train[num_cols] = h1n1_scaler.fit_transform(h1n1_train[num_cols])

seas_scaler = StandardScaler()
seas_train[num_cols] = seas_scaler.fit_transform(seas_train[num_cols])

h1n1_train_trans = h1n1_train
seas_train_trans = seas_train

In [12]:
categorical_features_indices = np.where(train.dtypes != float)[0]

# H1N1

## CatBoost and Optuna

In [13]:
X = h1n1_train_trans
y = h1n1_labels

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [15]:
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.metrics import roc_curve, roc_auc_score
import optuna

In [16]:
train_dataset = Pool(data=X_train,
                    label=y_train,
                    cat_features=categorical_features_indices)

In [17]:
def objective(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500]),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        'od_type' : "Iter",
        'od_wait' : 100,
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
         'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset,
            param,
            fold_count=5, 
            early_stopping_rounds=10,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [18]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=150)

[I 2021-07-12 08:06:39,005] A new study created in memory with name: no-name-7027df12-1cfb-4fd3-98da-abd91a984406


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 08:06:48,192] Trial 0 finished with value: 0.864903023372403 and parameters: {'iterations': 200, 'learning_rate': 0.25998666758670563, 'random_strength': 7, 'bagging_temperature': 7, 'max_bin': 5, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 7, 'max_depth': 3, 'l2_leaf_reg': 8.345387083873532e-06, 'one_hot_max_size': 12, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.864903023372403.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 08:07:34,479] Trial 1 finished with value: 0.8667626979331121 and parameters: {'iterations': 500, 'learning_rate': 0.2055856749271349, 'random_strength': 5, 'bagging_temperature': 1, 'max_bin': 6, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 8, 'max_depth': 10, 'l2_leaf_reg': 8.8771488946556, 'one_hot_max_size': 10, 'auto_class_weights': 'Balanced'}. Best is trial 1 with value: 0.8667626979331121.
[I 2021-07-12 08:07:37,418] Trial 2 finished with value: 0.8642582760804116 and parameters: {'iterations': 100, 'learning_rate': 0.29607919404355465, 'random_strength': 8, 'bagging_temperature': 2, 'max_bin': 5, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 4, 'max_depth': 2, 'l2_leaf_reg': 1.2877252667142128e-05, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.8667626979331121.
[I 2021-07-12 08:07:49,577] Trial 3 finished with value: 0.8533562281991152 and parameters: {'iterations': 200, 'learning_rate': 0.010397326520333542, '

...

[I 2021-07-12 16:47:13,560] Trial 146 finished with value: 0.8704011187226712 and parameters: {'iterations': 1200, 'learning_rate': 0.01858580641056655, 'random_strength': 2, 'bagging_temperature': 1, 'max_bin': 30, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 10, 'max_depth': 7, 'l2_leaf_reg': 10.921715804038323, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 141 with value: 0.8708605420659264.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 16:48:02,793] Trial 147 finished with value: 0.8699497967297087 and parameters: {'iterations': 1200, 'learning_rate': 0.019636490024241518, 'random_strength': 2, 'bagging_temperature': 1, 'max_bin': 30, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 10, 'max_depth': 7, 'l2_leaf_reg': 10.518035109311832, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 141 with value: 0.8708605420659264.
[I 2021-07-12 16:49:42,200] Trial 148 finished with value: 0.8583620738248163 and parameters: {'iterations': 1200, 'learning_rate': 0.001078790372508973, 'random_strength': 2, 'bagging_temperature': 0, 'max_bin': 30, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 10, 'max_depth': 7, 'l2_leaf_reg': 20.925578152424258, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 141 with value: 0.8708605420659264.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 16:50:10,092] Trial 149 finished with value: 0.8700216353205963 and parameters: {'iterations': 1200, 'learning_rate': 0.027294328437381707, 'random_strength': 1, 'bagging_temperature': 1, 'max_bin': 30, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 10, 'max_depth': 6, 'l2_leaf_reg': 6.696547294624169, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 141 with value: 0.8708605420659264.


In [19]:
trial = study.best_trial
final_model = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                          **trial.params)

final_model.fit(X_train, y_train)

final_h1n1_model = final_model
trial.params

{'iterations': 1200,
 'learning_rate': 0.00947709165733447,
 'random_strength': 1,
 'bagging_temperature': 0,
 'max_bin': 30,
 'grow_policy': 'Depthwise',
 'min_data_in_leaf': 10,
 'max_depth': 6,
 'l2_leaf_reg': 38.57722543123498,
 'one_hot_max_size': 100,
 'auto_class_weights': 'SqrtBalanced'}

In [20]:
params = trial.params
final_h1n1_model = CatBoostClassifier(cat_features=categorical_features_indices,
                                     verbose=False,
                                     **params).fit(h1n1_train_trans, h1n1_labels)

# Seasonal

## Catboost and Optuna

In [21]:
X = seas_train_trans
y = seas_labels

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [23]:
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.metrics import roc_curve, roc_auc_score
import optuna

In [24]:
train_dataset = Pool(data=X_train,
                    label=y_train,
                    cat_features=categorical_features_indices)

In [25]:
def objective(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500]),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        'od_type' : "Iter",
        'od_wait' : 100,
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
         'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset,
            param,
            fold_count=5, 
            early_stopping_rounds=10,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

In [ ]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=150)

[I 2021-07-12 16:51:02,078] A new study created in memory with name: no-name-2f6a6e58-440f-42e8-bc6e-595ec566f138


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 16:51:12,632] Trial 0 finished with value: 0.8586314270955565 and parameters: {'iterations': 200, 'learning_rate': 0.25998666758670563, 'random_strength': 7, 'bagging_temperature': 7, 'max_bin': 5, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 7, 'max_depth': 3, 'l2_leaf_reg': 8.345387083873532e-06, 'one_hot_max_size': 12, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8586314270955565.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 16:52:02,793] Trial 1 finished with value: 0.8578164334975481 and parameters: {'iterations': 500, 'learning_rate': 0.2055856749271349, 'random_strength': 5, 'bagging_temperature': 1, 'max_bin': 6, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 8, 'max_depth': 10, 'l2_leaf_reg': 8.8771488946556, 'one_hot_max_size': 10, 'auto_class_weights': 'Balanced'}. Best is trial 0 with value: 0.8586314270955565.
[I 2021-07-12 16:52:07,196] Trial 2 finished with value: 0.8574507580276862 and parameters: {'iterations': 100, 'learning_rate': 0.29607919404355465, 'random_strength': 8, 'bagging_temperature': 2, 'max_bin': 5, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 4, 'max_depth': 2, 'l2_leaf_reg': 1.2877252667142128e-05, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8586314270955565.
[I 2021-07-12 16:52:24,105] Trial 3 finished with value: 0.8422165888360776 and parameters: {'iterations': 200, 'learning_rate': 0.010397326520333542, '

...

[I 2021-07-12 18:53:58,016] Trial 143 finished with value: 0.8626573124571169 and parameters: {'iterations': 1000, 'learning_rate': 0.02053826903378038, 'random_strength': 4, 'bagging_temperature': 8, 'max_bin': 5, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 10, 'max_depth': 6, 'l2_leaf_reg': 5.980649624639963, 'one_hot_max_size': 12, 'auto_class_weights': 'Balanced'}. Best is trial 104 with value: 0.8633240486794842.


Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 18:55:34,022] Trial 144 finished with value: 0.8622979311604271 and parameters: {'iterations': 1000, 'learning_rate': 0.020112087887635167, 'random_strength': 4, 'bagging_temperature': 8, 'max_bin': 5, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 9, 'max_depth': 6, 'l2_leaf_reg': 1.178321291165751, 'one_hot_max_size': 12, 'auto_class_weights': 'Balanced'}. Best is trial 104 with value: 0.8633240486794842.
[I 2021-07-12 18:58:18,782] Trial 145 finished with value: 0.8602651478018901 and parameters: {'iterations': 1000, 'learning_rate': 0.011641960089661153, 'random_strength': 5, 'bagging_temperature': 8, 'max_bin': 5, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 10, 'max_depth': 7, 'l2_leaf_reg': 6.209335248639217, 'one_hot_max_size': 12, 'auto_class_weights': 'Balanced'}. Best is trial 104 with value: 0.8633240486794842.
[I 2021-07-12 18:58:33,789] Trial 146 finished with value: 0.853758733275531 and parameters: {'iterations': 100, 'learning_rate': 0.03590097688

Stopped by overfitting detector  (10 iterations wait)


[I 2021-07-12 18:59:24,974] Trial 147 finished with value: 0.8621145105922151 and parameters: {'iterations': 1000, 'learning_rate': 0.027024281378316056, 'random_strength': 4, 'bagging_temperature': 8, 'max_bin': 5, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 9, 'max_depth': 6, 'l2_leaf_reg': 19.566819769739897, 'one_hot_max_size': 1024, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 104 with value: 0.8633240486794842.


In [ ]:
trial = study.best_trial
final_model = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                          **trial.params)
trial.params

In [ ]:
params = trial.params
final_seas_model = CatBoostClassifier(cat_features=categorical_features_indices,
                                     verbose=False,
                                     **params).fit(seas_train_trans, seas_labels)

### Train Accuracy

In [ ]:
h1n1_train_data = train_test.copy()
h1n1_train_data[num_cols] = h1n1_scaler.transform(h1n1_train_data[num_cols])

seas_train_data = train_test.copy()
seas_train_data[num_cols] = seas_scaler.transform(seas_train_data[num_cols])

In [ ]:
y_predicted_h1n1 = final_h1n1_model.predict_proba(h1n1_train_data)[:,1].reshape(-1,1)
y_predicted_seas = final_seas_model.predict_proba(seas_train_data)[:,1].reshape(-1,1)

y_true = np.array(labels)

y_predicted = np.concatenate((y_predicted_h1n1, y_predicted_seas), axis=1)

In [ ]:
roc_auc_score(np.array(labels), y_predicted)

# Submission

In [ ]:
test = pd.read_csv('../Data/test_set_features.csv', index_col='respondent_id')
full_test = test.copy()

In [ ]:
num_cols = list(test.select_dtypes('number').columns)

cat_cols = [
    'race',
    'sex',
    'marital_status',
    'rent_or_own',
    'hhs_geo_region',
    'census_msa',
    'employment_industry',
    'employment_occupation'
]

ord_cols = [
    'age_group',
    'education',
    'income_poverty',
    'employment_status'
]


#Impute Test
for col in num_cols:
    test[col] = test[col].fillna(value=-1)


for col in (cat_cols+ord_cols):
    test[col] = test[col].fillna(value='None')

    
test['age_group'] = test['age_group'].map({
    '18 - 34 Years': 1,
    '35 - 44 Years': 2,
    '45 - 54 Years': 3,
    '55 - 64 Years': 4,
    '65+ Years': 5
})
    
test['education'] = test['education'].map({
    '< 12 Years': 1,
    '12 Years': 2,
    'Some College': 3,
    'College Graduate': 4,
    'None': -1
})

test['income_poverty'] = test['income_poverty'].map({
    'None': -1,
    'Below Poverty': 1,
    '<= $75,000, Above Poverty': 2,
    '> $75,000': 3
})

test['employment_status'] = test['employment_status'].map({
    'None': -1,
    'Unemployed': 1,
    'Employed': 2,
    'Not in Labor Force': 3
})

In [ ]:
test_h1n1 = test.copy()
test_seas = test.copy()

test_h1n1[num_cols] = h1n1_scaler.transform(test_h1n1[num_cols])
test_seas[num_cols] = seas_scaler.transform(test_seas[num_cols])

In [ ]:
y_h1n1 = final_h1n1_model.predict_proba(test_h1n1)[:,1].reshape(-1,1)
y_seas = final_seas_model.predict_proba(test_seas)[:,1].reshape(-1,1)

y_comb = np.concatenate((y_h1n1, y_seas), axis=1)

In [ ]:
y_comb

In [ ]:
results = pd.DataFrame(y_comb, columns=['h1n1_vaccine', 'seasonal_vaccine'], index=test.index)

submission = pd.concat([full_test, results], axis=1)
submission = submission[['h1n1_vaccine', 'seasonal_vaccine']]

In [ ]:
submission

In [ ]:
today = datetime.today().date()

submission.to_csv(f'../Submissions/Neural Network Submission {today}.csv')

In [ ]:
submission